# Load data

In [ ]:
from openprompt.data_utils import InputExample
from openprompt import PromptForClassification
from openprompt import PromptDataLoader
from openprompt.prompts import SoftTemplate
from openprompt.plms import load_plm
from openprompt.prompts import ManualVerbalizer
import pandas

# Load data
train_tsvreader = pandas.read_csv("train.tsv",sep='\t')
train_data      = []
# [TODO] 
# Load training data


valid_tsvreader = pandas.read_csv("dev.tsv",sep='\t')
test_data       = []
# [TODO] 
# Load testing data

# Define model and template

In [ ]:
plm, tokenizer, model_config, WrapperClass = load_plm('bert', 'bert-base-uncased')

template    = ''  # [TODO] Define a template
soft_tokens =     # [TODO] number of soft tokens
mytemplate  = SoftTemplate(model=plm, text=template, tokenizer=tokenizer,
                    num_tokens=soft_tokens, initialize_from_vocab=True)

classes = [ # There are two classes in Sentiment Analysis, one for negative and one for positive
    "0",
    "1"
]
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        "0": ["bad","terrible","disgusting","horrible"],
        "1": ["good", "wonderful", "great","excellent"],
    },
    tokenizer = tokenizer,
)

prompt_model =      #[TODO] Combine Template and Verbalizer into a PromptModel


# Define DataLoader

In [ ]:

train_dataloader = PromptDataLoader(
    #[TODO] 
)
valid_dataloader = PromptDataLoader(
    #[TODO] 
)


# Train and inference

In [83]:
from transformers import (
    AdamW,
)

optimizer_grouped_parameters = # [TODO] freeze parameter of encoder, only update soft tokens 

# Using different optimizer for prompt parameters and model parameters
optimizer = AdamW(optimizer_grouped_parameters, lr= # [TODO])

for epoch in range(# [TODO] number of epochs):
    # [TODO]